In [ ]:
import numpy as np
import plotly.graph_objects as go

# Parameters
L = 100                # Number of training samples
sigma_squared = 0.04    # Variance of noise
M = 3                 # Polynomial degree (model order)
eta = 0.3              # Learning rate 
Liter = 3000           # Number of iterations

# Synthetic data
np.random.seed(42)
t_train = np.random.uniform(0, 1, L)
y_train = np.sin(2 * np.pi * t_train) + np.random.normal(0, np.sqrt(sigma_squared), L)

# Polynomial basis function
def polynomial_basis(t, degree):
    return np.vstack([t**i for i in range(degree + 1)]).T

# Gradient descent
def gradient_descent(t, y, degree, eta, Liter):
    N = len(t)
    X = polynomial_basis(t, degree)
    W = np.random.randn(degree + 1)  # Initialize weights randomly
    mse_history = []

    for _ in range(Liter):
        y_pred = X @ W
        error = y_pred - y
        gradient = (2 / N) * X.T @ error
        W -= eta * gradient
        mse = np.mean(error**2)
        mse_history.append(mse)

    return W, mse_history

# Train the model
W, mse_history = gradient_descent(t_train, y_train, M, eta, Liter)

# MSE as a function of iterations
fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=list(range(1, Liter + 1)),
    y=mse_history,
    mode='lines',
    name='MSE during training'
))
fig1.update_layout(
    title="Mean Squared Error vs Iterations",
    xaxis_title="Iteration",
    yaxis_title="MSE",
    template="plotly_white"
)
fig1.show()

# Unseen data
t_test = np.arange(0, 1.01, 0.01)
y_test = np.sin(2 * np.pi * t_test)
X_test = polynomial_basis(t_test, M)
y_pred_test = X_test @ W

fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=t_test,
    y=y_test,
    mode='lines',
    name='True function y(t) = sin(2πt)',
    line=dict(color='blue')
))
fig2.add_trace(go.Scatter(
    x=t_train,
    y=y_train,
    mode='markers',
    name='Training data',
    marker=dict(color='red', size=8, opacity=0.7)
))
fig2.add_trace(go.Scatter(
    x=t_test,
    y=y_pred_test,
    mode='lines',
    name='Learned model',
    line=dict(color='green', dash='dash')
))
fig2.update_layout(
    title="Learned Model vs True Function",
    xaxis_title="Time t",
    yaxis_title="y(t)",
    template="plotly_white"
)
fig2.show()

# Varying L and observing performance
results = []
for L in range(10, 101, 10):
    t_train = np.random.uniform(0, 1, L)
    y_train = np.sin(2 * np.pi * t_train) + np.random.normal(0, np.sqrt(0.3), L)
    W, mse_history = gradient_descent(t_train, y_train, M, 0.1, 5000)
    results.append((L, mse_history[-1]))

# Extract results
L_values, mse_values = zip(*results)

# Plotting MSE vs L using Plotly
fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=L_values,
    y=mse_values,
    mode='lines+markers',
    name='Final MSE',
    marker=dict(size=8)
))
fig3.update_layout(
    title="Impact of Training Set Size L on Final MSE",
    xaxis_title="Training Set Size L",
    yaxis_title="Final MSE",
    template="plotly_white"
)
fig3.show()

In [54]:
import numpy as np
import plotly.graph_objects as go

# Parameters
L = 100                # Number of training samples
sigma_squared = 0.7    # Variance of noise
M = 9                  # Polynomial degree
eta = 0.3              # Learning rate for Gradient Descent
Liter = 3000           # Number of iterations for Gradient Descent

# Generate synthetic data
np.random.seed(42)
t_train = np.random.uniform(0, 1, L)
y_train = np.sin(2 * np.pi * t_train) + np.random.normal(0, np.sqrt(sigma_squared), L)

# Polynomial basis function
def polynomial_basis(t, degree):
    return np.vstack([t**i for i in range(degree + 1)]).T

# Moore-Penrose Pseudo-Inverse method
def compute_weights_mppi(t, y, degree):
    X = polynomial_basis(t, degree)
    pseudo_inverse = np.linalg.pinv(X)  # Compute Moore-Penrose Pseudo-Inverse
    W = pseudo_inverse @ y  # Calculate weights
    return W

# Gradient Descent function
def gradient_descent(t, y, degree, eta, Liter):
    N = len(t)
    X = polynomial_basis(t, degree)
    W = np.random.randn(degree + 1)  # Initialize weights randomly
    mse_history = []

    for _ in range(Liter):
        y_pred = X @ W
        error = y_pred - y
        gradient = (2 / N) * X.T @ error
        W -= eta * gradient
        mse = np.mean(error**2)
        mse_history.append(mse)

    return W, mse_history

# Compute weights using MPPI
W_mppi = compute_weights_mppi(t_train, y_train, M)

# Evaluate Gradient Descent weights
W_gd, _ = gradient_descent(t_train, y_train, M, eta, Liter)

# Evaluate on unseen data
t_test = np.linspace(0, 1, 100)
y_test = np.sin(2 * np.pi * t_test)
X_test = polynomial_basis(t_test, M)
y_pred_mppi = X_test @ W_mppi
y_pred_gd = X_test @ W_gd

# Plot predictions using Plotly
fig = go.Figure()

# True function
fig.add_trace(go.Scatter(
    x=t_test,
    y=y_test,
    mode='lines',
    name="True function y(t) = sin(2πt)",
    line=dict(color='blue')
))

# Training data
fig.add_trace(go.Scatter(
    x=t_train,
    y=y_train,
    mode='markers',
    name="Training data",
    marker=dict(color='red', size=8, opacity=0.7)
))

# MPPI model
fig.add_trace(go.Scatter(
    x=t_test,
    y=y_pred_mppi,
    mode='lines',
    name="MPPI model",
    line=dict(color='green', dash='dash')
))

# Gradient Descent model
fig.add_trace(go.Scatter(
    x=t_test,
    y=y_pred_gd,
    mode='lines',
    name="Gradient Descent model",
    line=dict(color='orange', dash='dot')
))

# Update layout
fig.update_layout(
    title="Comparison of MPPI and Gradient Descent Models",
    xaxis_title="Time t",
    yaxis_title="y(t)",
    template="plotly_white"
)

# Show the plot
fig.show()

In [62]:
import numpy as np
from sklearn.model_selection import KFold
import plotly.graph_objects as go

# Parameters
L = 100                 # Number of samples
sigma_squared = 0.09    # Noise variance
model_orders = range(1, 12)  # Model orders from 1 to 11
np.random.seed(42)

# Generate dataset
t_data = np.random.uniform(0, 1, L)
y_data = np.sin(2 * np.pi * t_data) + np.random.normal(0, np.sqrt(sigma_squared), L)

# Polynomial basis function
def polynomial_basis(t, degree):
    return np.vstack([t**i for i in range(degree + 1)]).T

# Moore-Penrose Pseudo-Inverse method
def compute_weights_mppi(t, y, degree):
    X = polynomial_basis(t, degree)
    pseudo_inverse = np.linalg.pinv(X)  # Compute Moore-Penrose Pseudo-Inverse
    W = pseudo_inverse @ y  # Calculate weights
    return W

# K-Fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
train_rmse_per_order = []
test_rmse_per_order = []

# Evaluate RMSE for each model order
for M in model_orders:
    fold_train_errors = []
    fold_test_errors = []
    for train_index, test_index in kf.split(t_data):
        t_train, t_test = t_data[train_index], t_data[test_index]
        y_train, y_test = y_data[train_index], y_data[test_index]
        
        # Train model using MPPI
        W = compute_weights_mppi(t_train, y_train, M)
        
        # Evaluate model
        X_train = polynomial_basis(t_train, M)
        X_test = polynomial_basis(t_test, M)
        
        y_train_pred = X_train @ W
        y_test_pred = X_test @ W
        
        # Compute RMSE
        train_rmse = np.sqrt(np.mean((y_train - y_train_pred)**2))
        test_rmse = np.sqrt(np.mean((y_test - y_test_pred)**2))
        
        fold_train_errors.append(train_rmse)
        fold_test_errors.append(test_rmse)
    
    # Store average RMSE for this model order
    train_rmse_per_order.append(np.mean(fold_train_errors))
    test_rmse_per_order.append(np.mean(fold_test_errors))

# Plot RMSE vs Model Order using Plotly
fig = go.Figure()

# Add Training RMSE
fig.add_trace(go.Scatter(
    x=list(model_orders),
    y=train_rmse_per_order,
    mode='lines+markers',
    name="Training RMSE",
    line=dict(color='blue'),
    marker=dict(size=8)
))

# Add Test RMSE
fig.add_trace(go.Scatter(
    x=list(model_orders),
    y=test_rmse_per_order,
    mode='lines+markers',
    name="Test RMSE",
    line=dict(color='red'),
    marker=dict(size=8)
))

# Update layout
fig.update_layout(
    title="Training and Test RMSE vs Model Order (1 to 11)",
    xaxis=dict(title="Model Order"),
    yaxis=dict(title="Root Mean Square Error (RMSE)"),
    template="plotly_white"
)

# Show plot
fig.show()

# Optimal model order
optimal_model_order = model_orders[np.argmin(test_rmse_per_order)]
print(f"Optimal model order based on minimum Test RMSE: {optimal_model_order}")


Optimal model order based on minimum Test RMSE: 5
